# General

Online store Ice, which sells video games all over the world wants to determine whether a game succeeds or not. This will allow Ice to spot potential big winners and plan advertising campaigns.

We will analyze historical data and provide recomendations how to plan an advert campaign for the next year.

We will start by preparing and cleaning the data.

We will then analyze the data and look for distributions and patterns, find the winning scenarios, display charts to demonstrate our case.

We will continue with creating a user profile for each region and test 2 hypotheses:

1. *Average user ratings of the Xbox One and PC platforms are the same.*

2. *Average user ratings for the Action and Sports genres are different.*


In [ ]:
# Loading all the libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
from scipy import stats 
import matplotlib_inline.backend_inline
import math
import random

# If error in "import sidetable", please uncomment the following line:
!pip install sidetable

import sidetable

from google.colab import drive
drive.mount('/content/drive')

### These lines will make the graphs much crisper

if isinstance('svg', str):
   ipython_format = ['svg']
   matplotlib_inline.backend_inline.set_matplotlib_formats(*ipython_format)

In [ ]:
# Load the data files into raw_data df.

try:
  raw_df = pd.read_csv("/content/drive/MyDrive/Practicum/Module_1/Integrated Project_1/games.csv", )
except: 
  raw_df = pd.read_csv("/datasets/games.csv",)



Let's take a first look at our file:


In [ ]:
raw_df.sample(5)


In [ ]:

print("*********************************************")
print("****************** ＩＮＦＯ ******************")
print("*********************************************")
print(" ")
raw_df.info()
print(" ")
print("*********************************************")
print("**************** NANS COUNT *****************")
print("*********************************************")
print(" ")
raw_df.stb.missing(style=True)


In [ ]:
print("*********************************************")
print("*********     NUMERIC STATS      ************")
print("*********************************************")
print(" ")
raw_df.describe()

In [ ]:
print("*********************************************")
print("*********     OBJECT STATS      *************")
print("*********************************************")
print(" ")
raw_df.describe(include=['object'])

In [ ]:
# searching for duplicates 

print("There are", raw_df.duplicated(subset=['Name']).sum(), "duplicates in Name colums.")


Running our first lines of code we notice several things:

This is 11X16715 dataset with object and float columns. The dataset has several issues worth mentioning: 

1. User_Score column is dtype object (should be float).

2. **Missing values in**

 * Critic_Score	(8,578)	
 * Rating	(6,766)	
 * User_Score	(6,701)
 * Year_of_Release	(269)
 * Name	(2)	
 * Genre	(2)

3. Column names should be lowercase.

4. 50% of the games are from 2007 and older. 

5. Multiple instances of "tbd" str in user_score. It prevents us from using it as floats.


Let's start solving these problems, first by renaming the columns to lowercase:


In [ ]:
raw_df.columns = raw_df.columns.str.lower()

# Check:
raw_df.columns

 Solved. Now let's think what to do with the Nans:
 

In [ ]:
print("Since we have ", len(raw_df.name.unique()), "unique game names in", len(raw_df), "instances.")
print("will it be possible to asign the",raw_df.user_score.isna().sum(), "nans in user_score and", raw_df.critic_score.isna().sum(), "of critic_score nans?")


In many cases we have the same game repeating again and again on different platforms. Let's check if by grouping the instances by name and year_of release we will also get the same scores and ratings. 


In [ ]:
# Let's check a random game from a list, if you get less than 1 instance, run this cell again

# Let's choose instances that have ratings and name duplicates
proper_instances = raw_df[(raw_df.rating.notna()) & raw_df.name.duplicated() & raw_df.year_of_release.duplicated()]

# Let's choose and printa random game.
random_name = proper_instances.name.iloc[random.randrange(0,100)]
random_game = proper_instances[proper_instances.name == random_name]
random_game

**My Comment. v.1 DONE.**

By runnung the above cell several times we can see that although the name of the game and the year of the release are the same, the critic score and the user score are **platform dependent** and therefore **different**. 

Ratings, on the other hand are the same across all platforms, since the games are rated by the Entertainment Software Rating Board (ESRB).

**It means that we SHOULD NOT blindly asign scores based on names and years, but we CAN use it to asign ratings.**

Let's do this.

In [ ]:

# Let's drop all the Nans in ratings:

na_ratings_dropped = raw_df.dropna(subset =["rating"])
na_ratings_dropped.sample(5)

# Let's drop the duplicates of name and year_of_release, since all platforms got the same rating per game.

duplicates_dropped = na_ratings_dropped.drop_duplicates(subset=['name', "year_of_release"], keep = "first") 

# We are left with unique name+year instances.:

duplicates_dropped.sample(5)


In [ ]:
# Let's check that we don't have duplicates (games that have the same name AND year)

print("We have", duplicates_dropped.duplicated(subset = ["name", "year_of_release"]).sum(), "duplicates in this df.")


In [ ]:
# Let's create two dicts:

names_years_dict = duplicates_dropped.set_index('name')['year_of_release'].to_dict()

names_ratings_dict = duplicates_dropped.set_index('name')['rating'].to_dict()

In [ ]:
# Now let's create a new df called nans_in_ratings:

nans_in_rating = raw_df[raw_df.rating.isna()]

# Check:
nans_in_rating.sample(5)


In [ ]:
# Let's create a list:
restored_ratings = []

# Error handler
pd.options.mode.chained_assignment = None

# And now let's asign ratings to the instances that are missing their ratings:

for key, value in names_years_dict.items():
    nans_in_rating[(nans_in_rating.name == key) & (nans_in_rating.year_of_release == value)]["rating"] = names_ratings_dict[key]
    restored_ratings.append(names_ratings_dict[key])

nans_in_rating["restored_ratings"] =  pd.Series(restored_ratings)


In [ ]:
print("We have succesfully restored", nans_in_rating.restored_ratings.notna().sum(), "ratings!" )

In [ ]:

restored = nans_in_rating.drop(["rating"], axis = 1)

restored.rename(columns = {"restored_ratings" : "rating"}, inplace = True)

restored.sample(3)

In [ ]:

# Let's run a check
clean_df =pd.merge(raw_df, restored["rating"], left_index=True, right_index=True, how = "outer" )
print ("We have", clean_df.rating_x.notna().sum(), "valid instances in ratings_x and", clean_df.rating_y.notna().sum(), "in ratings_y."  )

# Let's unite the columns
clean_df["rating"] = clean_df.apply(lambda x: x['rating_y'] if pd.isnull(x['rating_x']) else x['rating_x'], axis=1)
print("We moved the ratings from y to x. Now we have",  clean_df.rating.notna().sum(), "valid instances ratings in column rating.")

# Let's get rid of x and y.
clean_df.drop(["rating_x", "rating_y"], axis = 1, inplace =True)


print(" ")
clean_df.sample(5)

#### FIX :  Now let's fix the remaining Nans in our rating column by asigning "NA" value to the nans.   

In [ ]:
clean_df.rating.fillna('NA', inplace=True)

In [ ]:
clean_df.stb.missing(style=True)

The rating column nans dropped from 40.4% to 29%.  We have successfully restored 11% of the ratings. But can we do the same thing with the critic_score and user_score? Yes and No. 

As already mentioned, unlike the ratings that are published by external body (ESRB), the critic score and user score are **platform specific**. Each platform has it's own audience, region of influence, number of players e.c. 

Arithmetically calculating the mean of the user_score of a game X across all platforms and simply asigning that number to a Nan -  wouldn't be much of a "cleaning of a data",since we would be ignoring all the factors involved. 

We advise against such practice and encourage to work with a quality data. It might be a small sample of the dataset, but it's values are not polluted by our "calculations", preconceptions and biases. **When analyzing data we should analyze DATA, and nothing else.**

Let's see how many instances are still missing ALL THREE tables (user_score, critic_score, rating).

"Tbd" in user_score column stands for "to be determined". At first we might think that this is an indication of a relatively new games, for which there is no user_score yet. But if we check the distribution of the tbds across the years we can see that this is not the case:

In [ ]:

raw_df[raw_df.user_score =="tbd"]["year_of_release"].value_counts()


Some of the games are very old, and still missing the user_score. Meaning that the *tbd* is more of a **placeholder for unknown data**. User scores, as already mentioned before, can not be "calculated". Any mean/median will only skew the data. 

What we can restore is the Nans in the *year of release* column:

In [ ]:

# Let's clean the years and save in new df
na_years_dropped = raw_df.dropna(subset =["year_of_release"])

# Let's drop the duplicates of name + year.
duplicated_years_dropped = na_years_dropped.drop_duplicates(subset=['name', "year_of_release"], keep = "first") 

# Let's create a new name_years dict and save names/years paires.
new_name_year_dict =  duplicated_years_dropped.set_index('name')['year_of_release'].to_dict()


Now let's create the dataset we want to "heal", to restore the years:

In [ ]:
# Let's create a data with nans in year_of_release
nans_in_year_of_release = raw_df[raw_df.year_of_release.isna()]
nans_in_year_of_release.sample(5)

Now lets' asign the missing years from the dictionary.

In [ ]:

# Let's create a list:
restored_years = []

# Error handler
pd.options.mode.chained_assignment = None

# And now let's asign years to the instances that are missing their years:

for key, value in new_name_year_dict.items():
    nans_in_year_of_release[nans_in_year_of_release.name == key]["year_of_release"] = new_name_year_dict[key]
    restored_years.append(new_name_year_dict[key])

nans_in_year_of_release["restored_years"] =  pd.Series(restored_years)

print("We have succesfully restored", nans_in_year_of_release.restored_years.notna().sum(), "instances.")

Our instances are stored in 
*nans_in_year_of_release/restored_years* column. Let's move it to the main df. 

In [ ]:

restored_years = nans_in_year_of_release.drop(["year_of_release"], axis = 1)

restored_years.rename(columns = {"restored_years" : "year_of_release"}, inplace = True)

restored.sample(3)


Let's merge the restored data with our clean_df in a new clean_df_2.

In [ ]:

# Let's run a check
clean_df_2 =pd.merge(clean_df, restored_years["year_of_release"], left_index=True, right_index=True, how = "outer" )
print ("We have", clean_df_2.year_of_release_x.isna().sum(), "nans instances in year_of_release_x and", clean_df_2.year_of_release_y.notna().sum(), " restored years in year_of_release_y."  )

# Let's unite the columns
clean_df_2["year_of_release"] = clean_df_2.apply(lambda x: x['year_of_release_y'] if pd.isnull(x['year_of_release_x']) else x['year_of_release_x'], axis=1)
print("We moved the ratings from y to x. Now we only have",  clean_df_2.year_of_release.isna().sum(), "nans in column year_of_release.")

# Let's get rid of x and y.
clean_df_2.drop(["year_of_release_x", "year_of_release_y"], axis = 1, inplace =True)

print(" ")
clean_df_2.sample(5)

Great! We succesfully restored 175 missing years.

Let's create a new column with the total sales (the sum of sales in all regions) for each game and put these values in a separate column.

In [ ]:
clean_df_2["total_sales"] = clean_df_2[["na_sales","eu_sales", "jp_sales","other_sales"]].sum(axis = 1)
clean_df_2.sample(5)

# Analyzing the data

### Question: **How many games were released in different years. Is the data for every period significant?**

In [ ]:
clean_df_2.stb.freq(["year_of_release"], style=True, cum_cols = False)

(*multiple zeros in years are due to the factr that we did not converted the years to ints, since we still have nans in our data)*. 

The table above shows the distribution of games across the years. From 1980 to 2016 the most "hot" year in the game industry was 2008 with 1,448 (8.41%) games. In 1980 - there were only 9 games. 

Just by looking at the numbers we can see an uptrend dinamit that reached it's peak in 2008-2009. The first 14 years (from 1980 to 1994) have less than 1% of number of games per year, whereas between 2005  - 20010 the ranges are between 5.7% - 8.7%.

Let's show this on a graph.

To do so we must first drop nans the years column as it will cause are erros in the next step.


In [ ]:
# New df with no nans in years
clean_df_3 = clean_df_2[clean_df_2.year_of_release.notna()]

print("Now, we have", clean_df_3.year_of_release.isna().sum(), "nans in years column.")
print(" ")
# Converting object to ints
clean_df_3.year_of_release = clean_df_3.year_of_release.astype("int")


In [ ]:
print("Let's see that table again:")
print(" ")

df=clean_df_3[['year_of_release','name']].groupby(['year_of_release']).count().sort_values(by='name', ascending=False).reset_index()
df.head(5)

Let's see how it looks like on the graph: 

In [ ]:

fig,ax=plt.subplots(figsize=(12,6))
ax.vlines(x=df.year_of_release, ymin=0, ymax=df.name, color='purple', alpha=0.7, linewidth=2)
ax.scatter(x=df.year_of_release, y=df.name, s=75, color='black',alpha=0.7)
plt.xlabel("YEARS")
plt.ylabel("NUMBER OF GAMES PER YEAR")
plt.title("Distribution of games per year between 1980 and 2016")

ax.set_xticks(df.year_of_release)
ax.set_xticklabels(df.year_of_release,rotation=90)
# ax.axhline(y=300)

for row in df.itertuples():
    ax.text(row.year_of_release, row.name+30, s=row.name)

plt.show()

The dinamics we mentioned earlier is clearly shown on the graph. 



### Question: **How sales varied from platform to platform. Choose the platforms with the greatest total sales and build a distribution based on data for each year.**

In order to answer this question, let's choose top 5 game sellers in the industry, based on the total sales.  

In [ ]:
# First by grouping on names and summing the total sales.  
unique_games = clean_df_2.groupby("platform").agg({"total_sales" : "sum"})

# Choosing the top 5 total sellers.
unique_games.nlargest(5,["total_sales"])

Let's recheck this with another method.

In [ ]:
df = clean_df_3[["platform", "total_sales"]].groupby("platform").sum().sort_values(by = "total_sales").reset_index()

df.nlargest(5, "total_sales")

Yep. Same guys at the top 5. Let's normalize the data somehow. We can zscore the data or min_max it between 0 and 1. 

Let's zscore:

In [ ]:
# Zscoring total_sales
df['zscored_total_sales']=(df['total_sales']- df['total_sales'].mean()) / df['total_sales'].std()

# To plot it, let's create a new column with two colors:
df['color']=['black' if x<0 else 'green' for x in df['zscored_total_sales']]

# Let's add a column plot it and see what can we learn:
plt.figure(figsize=(10,8))
plt.hlines(y=df.platform,xmax=df.zscored_total_sales, xmin=0,color = df.color, alpha=0.8, linewidth=10)
plt.title("Normalized sales per platform.")
plt.ylabel("Platforms")
plt.xlabel("Distribution of sales per platform")
plt.show()


Our top 5 platforms are at the top of the green bars. The point that seperates the color is the mean. The green bars are total sales above the mean. The black bars beneath the mean.

If we add to our top 5 list number 6 (platform PS), we can definetivly say that these 6 are the game industry showrunners on total, across the years.  

**Find platforms that used to be popular but now have zero sales.**

In [ ]:
df=clean_df_3.groupby(['platform'])['year_of_release'].agg(['max']).sort_values("max").head(15)

print(" The following list shows the last ~living year~ of 15 platforms. \n",
"Meaning that the latest game on this platform was published in that year.")
print(" ")
df

**How long does it generally take for new platforms to appear and old ones to fade?**

Let's calculate.

In [ ]:
# Creating new df with min max of all platforms. 
df=clean_df_3.groupby(['platform'])['year_of_release'].agg(["min", "max"]).reset_index()

# Choosing only the platforms that were "born" on 2006 and later and "died" before 2016 (We don't want a living platforms in our calculation)
df = df[df["min"] >= 1990] 

# Saving the age as life_span
df["life_span"] = df["max"] - df["min"]

print("The averaged mean/median lifespan of 1990  - 2016 born platforms is", round((df.life_span.mean() + df.life_span.median()) / 2, 2), "years.")



This 7.7 number might assist us in our further decisions - for example, we know how "farther down the road" of it's market life our platform is, since we know the avr year of platform's life. 

Another thing that this number might assist us with is filtering our data with 7.7 years. For example we might choose to look ONLY at the data from the platforms that are within this range of years (i.e. 2008+) and regard it a a logical subset, unlike subsets based on a more "intuitive" approach.

Let's see if there's a corr() between life_span and total sales:

In [ ]:
df2=clean_df_3.groupby(['platform'])["total_sales"].agg(["min", "max"]).reset_index()

print("There's", int(100*df2["max"].corr(df["life_span"])),"% correlation between the life_span of the platform and its total revenue.")



Well, no surprise. 

### **Which platforms are leading in sales? Which ones are growing or shrinking? Select several potentially profitable platforms.**


In order to answe this question let's put these life spans on a graph.

In [ ]:
# Creating new df with games from 1980 and up
df=clean_df_3[clean_df_3.year_of_release >= 1980]

df=df.groupby(['platform','year_of_release'])['total_sales'].sum().reset_index().sort_values(by='year_of_release')

plt.figure(figsize=(10,6))
ax = sns.lineplot(data=df, x="year_of_release", y="total_sales",hue="platform")
ax.legend(bbox_to_anchor=(1.05,1.0),loc='best')
plt.title("Platforms performance over the years.")
plt.ylabel("Games")
plt.xlabel("Years")
ax.set_facecolor('black')
# ax.set_facecolor((1.0, 0.47, 0.42))
plt.show()

By just looking on the total period - from 1980 to 2016, we can definitely see the platforms "born" and "die". As we calculated previously the avr lifespan of a platform is 7.7 yeas. So let's subset the data from the last 8 years.

In [ ]:
df=clean_df_3[clean_df_3.year_of_release >= 2013]

df=df.groupby(['platform','year_of_release'])['total_sales'].sum().reset_index().sort_values(by='year_of_release')

plt.figure(figsize=(10,6))
ax = sns.lineplot(data=df, x="year_of_release", y="total_sales",hue="platform")
ax.legend(bbox_to_anchor=(1.05,1.0),loc='best')
plt.title("Platforms performance over the years.")
plt.ylabel("Games")
plt.xlabel("Years")
ax.set_facecolor('black')
# ax.set_facecolor((1.0, 0.47, 0.42))
plt.show()


**To the degree that the numbers for the year 2016 are final**, we can not ignore the overall **downtrend** in the sales, not just in the revenues of the top sellers, **but the game industry as a whole**. 

This downtrend might be explained by the fact that the users shift toward ONLINE and MOBILE app games, that are platform "free". In addition, in the last years, new trends have emerged  that did not exist previously, like  games on social mediaand so on.

Back to the question: the "profitiability" of a platform **from our data** can only be calculated to the degree that the platform has sales... But sales, as we know, doesn't mean revenues. In addition, such "profitability" says little about the overall position of the platform on the market. As clearly seen in the graph in 2016 all platforms UNDERPERFORM their previous TWO years. A clear indication of industry downtrend (if measured by in total sales, given our data for 2016 is complete).



Let's see this on a heat map:

In [ ]:
relevant_years=clean_df_3[clean_df_3.year_of_release>=2013]
relevant_years

df=pd.pivot_table(relevant_years, index='year_of_release',columns='platform',values='total_sales',aggfunc='sum',fill_value=0)
trends = (df-df.shift(+1)).T


plt.figure(figsize=(13,9))
sns.heatmap(trends, cmap='RdBu_r')
plt.title("Heatmap of platforms revenues")
plt.show()

The heatmap above clearly demonstrates the "relevance" of a platform. We want to choose the most profitable platforms in the recent years - on the graph these platforms have the most brownish-red colors. We identified 2 platforms that met this criteria:  XOne and PS4. Both are in the downtrend (as noted previously) and their total sales are shrinking but (since we are pushed to the wall and must choose) these are our top 2 platforms. 

### **Build a box plot for the global sales of each game, broken down by platform. Are the differences in sales significant? What about average sales on various platforms? Describe your findings.**

In [ ]:
# Let's take only the relevant period of years.
data_new = clean_df_3[clean_df_3.year_of_release >= 2013]

# and group it by platform and name, then summing the total sales
grouped = data_new.groupby(['platform','name'])['total_sales'].sum().reset_index()


In [ ]:
# let's plot it.

import seaborn as sns

plt.figure(figsize=(13,10))
sns.boxplot(x='platform',y='total_sales',data=grouped, showfliers = False)
plt.title("Boxplot of global sales of each game/per platform, not incl outliers ")
plt.show()

The box plot above demonstrates the historical state of the industry (bt 2013-2016). What we can learn is the overall distribution of sales on the platforms. Some platforms are "dying" with diminishing sales while others still demonstrate very strong performance with very high medians of sales.  

In [ ]:
print("The avr total sales across all relevant platforms is :", round(grouped.total_sales.mean(),2))

#### **Take a look at how user and professional reviews affect sales for one popular platform (you choose). Build a scatter plot and calculate the correlation between reviews and sales. Draw conclusions.**
#### **Keeping your conclusions in mind, compare the sales of the same games on other platforms**.

- build a function and iterate over it 

To answer this question let's first clean our data a little:

In [ ]:
clean_df_4 = data_new[(data_new.user_score.notna()) & (data_new.user_score != "tbd") & (data_new.critic_score.notna()) ]
clean_df_4.user_score = clean_df_4.user_score.astype(float)
clean_df_4.info()

Now let's build a func to do the hardlifting:

In [ ]:
# Function that calculates the corr btw scores and total sales. 

def score_effect(platform,score_type):
    data=clean_df_4[(clean_df_4.platform == platform) & (clean_df_4[score_type]).notna()][['total_sales', score_type]]
    correlation = data[score_type].corr(data.total_sales)
    return correlation

plats = clean_df_4.platform.unique().tolist()
types = ["user_score", "critic_score"]
corr_list = []

for p in plats:
  for t in types:
    corr_list.append([p, t, round(score_effect(p,t), 2)])

Let's display our findings:

In [ ]:
new_df = pd.DataFrame(data = corr_list, columns = ["platform", "score_type", "corr"])
new_df.sort_values("corr", ascending = False)

Our new df shows that critic scores, on most platforms, have higher corr with total sales than the user_scores. Probably because they review the games before they are distributed. Let's see this on a graph for platform : 

In [ ]:
game = clean_df_4.query('platform == "PS4"')

sales_ratings_scatter = game.pivot_table(index ="total_sales", values = ["critic_score", "user_score"])

pd.plotting.scatter_matrix(sales_ratings_scatter, figsize = (8,8))

plt.title("Influence of critics on sales in PS4")

plt.show()

print(" ")

game.corr()



The table graph and the table demonstrate a clear corr btw critics and sales. The highier the critic's score - the highier the score. This is hardly a linear correlation but nevertheless it exists and should not be ignored. 

**Take a look at the general distribution of games by genre. What can we say about the most profitable genres? Can you generalize about genres with high and low sales?**

In [ ]:
# Let's take only the relevant period of years.
data_new = clean_df_3[clean_df_3.year_of_release >= 2008]

# and group it by platform and name, then summing the total sales
grouped = data_new.groupby(['genre'])['total_sales'].mean().reset_index()

plt.figure(figsize=(13,8))
sns.boxplot(x='genre',y='total_sales',data=grouped)
plt.title("Total sales by genre, not incl outliers ")
plt.show()

Table above clearly demonstrates that on mean of total sales across all platform the winning genre is "shooter" and "platformers". Adventure games seems to have the lowest total sales, followed by a puzzle and strategy games. Seems like people want to be entertained and not develop their brains...

### Step 4. Create a user profile for each region
For each region (NA, EU, JP), determine:

The top five platforms. Describe variations in their market shares from region to region.

The top five genres. 

Explain the difference.

Do ESRB ratings affect sales in individual regions?

In [ ]:
# Let's calculate the top selling platforms in eu, jp, na regions:

NA=clean_df_4.groupby(['platform'])['na_sales'].mean().reset_index().sort_values(by='na_sales',ascending=False).head()
print("The Top platforms in NA region are:", NA.platform.tolist()) 

JP=clean_df_4.groupby(['platform'])['jp_sales'].mean().reset_index().sort_values(by='jp_sales',ascending=False).head()
print("The Top platforms in JP region are:", JP.platform.tolist()) 

EU=clean_df_4.groupby(['platform'])['eu_sales'].mean().reset_index().sort_values(by='eu_sales',ascending=False).head()
print("The Top platforms in EU region are:", EU.platform.tolist()) 


In [ ]:
NA.platform.value_counts().plot.pie(y='type', figsize=(6, 6), autopct='%1.1f%%',title = "Best selling platforms in NA")
plt.show()

EU.platform.value_counts().plot.pie(y='type', figsize=(6, 6), autopct='%1.1f%%',title = "Best selling platforms in EU")
plt.show()


JP.platform.value_counts().plot.pie(y='type', figsize=(6, 6), autopct='%1.1f%%',title = "Best selling platforms in JP")
plt.show()


In [ ]:
# Let's find the top five genres in our regions:

NA=clean_df_4.groupby(['genre'])['na_sales'].mean().reset_index().sort_values(by='genre',ascending=False).head()
print("The Top genres in NA region are:", NA.genre.tolist()) 

JP=clean_df_4.groupby(['genre'])['jp_sales'].mean().reset_index().sort_values(by='genre',ascending=False).head()
print("The Top genres in JP region are:", JP.genre.tolist()) 

EU=clean_df_4.groupby(['genre'])['eu_sales'].mean().reset_index().sort_values(by='genre',ascending=False).head()
print("The Top genres in EU region are:", EU.genre.tolist()) 

Seems like people change but not in nature. The same exact genres, in the same exact order...

Let's check the corr between the genres and the sales in these regions... In order to do this let's asign numbers to the genres and use them insted of "genre" table. 

In [ ]:

NA=clean_df_4.groupby(['genre'])['na_sales'].mean().reset_index().sort_values(by='genre',ascending=False).head().reset_index()
NA["int_label"] = NA.index + 1
print("The genre corr in NA region is:", round(NA.int_label.corr(NA.na_sales),2))

JP=clean_df_4.groupby(['genre'])['jp_sales'].mean().reset_index().sort_values(by='genre',ascending=False).head().reset_index()
JP["int_label"] = JP.index + 1
print("The genre corr in JP region is:", round(JP.int_label.corr(JP.jp_sales),2)) 

EU=clean_df_4.groupby(['genre'])['eu_sales'].mean().reset_index().sort_values(by='genre',ascending=False).head().reset_index()
EU["int_label"] = EU.index + 1
print("The genre corr in EU region is:", round(EU.int_label.corr(EU.eu_sales),2))




Seems like in JP and EU, there is very significant correlation between the genre and the sales. 

Let's see if ratings affect the sales in these regions. We are doing the same exact trick with the obj column of rating: we are asigning it a number.

In [ ]:
NA=clean_df_4.groupby(['rating'])['na_sales'].mean().reset_index().sort_values(by='rating',ascending=False).head().reset_index()
NA["int_label"] = NA.index + 1
print("The rating/sales corr in NA region is:", round(NA.int_label.corr(NA.na_sales),2))

JP=clean_df_4.groupby(['rating'])['jp_sales'].mean().reset_index().sort_values(by='rating',ascending=False).head().reset_index()
JP["int_label"] = JP.index + 1
print("The rating/sales corr in JP region is:", round(JP.int_label.corr(JP.jp_sales),2)) 

EU=clean_df_4.groupby(['rating'])['eu_sales'].mean().reset_index().sort_values(by='rating',ascending=False).head().reset_index()
EU["int_label"] = EU.index + 1
print("The rating/sales corr in EU region is:", round(EU.int_label.corr(EU.eu_sales),2))



The corr of ratings and sales in EU is the highest -  56%!

In [ ]:
for region in ["na_sales", "eu_sales", "jp_sales"]:
  mean=clean_df_4.groupby(['rating'])[region].mean().reset_index().sort_values(by=region,ascending=False).reset_index().head(5)
  total=clean_df_4.groupby(['rating'])[region].sum().reset_index().sort_values(by=region,ascending=False).reset_index().head(5)
  winning_ratings = [x for x in mean.rating.to_list() if x in total.rating.to_list()]
  print(region, "best selling ratings :", winning_ratings)


Our analysis shows that na, eu and jp regions have identical group of top 5 (4 actually, NA is not a rating) ratings. 

### Step 5. Test the following hypotheses:

#### H*o*, null: Average user ratings of the Xbox One and PC platforms are the same.

#### H*a*, null: Average user ratings of the Xbox One and PC platforms are not the same.

##### Significance level: alpha = 0.08

##### n = 120 samples from each group

In [ ]:
from scipy import stats as st

pc = clean_df_4.query("platform == 'PC'")
xone =  clean_df_4.query("platform == 'XOne'")

# Let's set alpha level
alpha = 0.08

results = st.ttest_ind(pc["user_score"].sample(n=120), xone["user_score"].sample(n=120),  equal_var = False)

if results.pvalue < alpha:
  print("We reject the null hypothesis, because:")
else:
  print("We can't reject the null hypothesis, because:")

print("Our pvalue is", round(results.pvalue,4))
print("PC user_score mean is", pc.user_score.mean())
print("Xbox_one user_score mean is", xone.user_score.mean())



My Comment. v. 1 : DONE

#### Hypotheses 2 

#### H*a*, alternative: Average user ratings for the Action and Sports genres are different.

#### H*o*, alternative: Average user ratings for the Action and Sports genres are the same.

This means that our null H is that the ratings for Action and Sports genres are the same. 

Significance level: alpha = 0.08

n = 150 samples from each group


In [ ]:
from scipy import stats as st

action = clean_df_4.query("genre == 'Action'")
sports =  clean_df_4.query("genre == 'Sports'")

# Let's set alpha level
alpha = 0.08

results = st.ttest_ind(pc["user_score"].sample(n=100), xone["user_score"].sample(n=100),  equal_var = False)

if results.pvalue < alpha:
  print("We reject the null hypothesis, because:")
else:
  print("We can't reject the null hypothesis, because:")

print("Our pvalue is", round(results.pvalue,2))
print("Sports genre user_score mean is", sports.user_score.mean())
print("Action genre user_score mean is", action.user_score.mean())


We defined our alpha on 8%, (5% to 8% are the most commonly used values for alpha for rejecting the null hypotheses.) Running the experement several times we can state the following:

We can't regect the null H1 that the average user ratings of the Xbox One and PC platforms are the same because their pvalue is 25% (our alpha is 8%)

We cannot reject the null H2 hypotheses that the average user ratings for the Action and Sports genres are the same because our pvalue is 17%. Therefore we reject our original, alternative hypotheses. 

Conclution:

We started our analysis by getting ourselves familiar wt the dataset. By calling a few info functions we noticed object columns that might cause us problems later on. Therefore we:

* lowercased the column names,
* Restored  1910 missing ratings based on the names of the games and year of release.
* Explained the distribution of tbds in the user score column.
* Restored 175 instances of missing year_of_release using the names of the games. 
* Calculated total sales column
* Analyzed the data by answering the questions in the taskand checking the hypotheseses proposed. 

Our overall conclution might be summerized in the following:

* Importance of clearly defined criteria for data gathering: parameters must be relevant (in our example 50% of the data was too old).
* The data must be complete (we proposed our suspicion that the data for 2016 sales is not complete.)
* Game store must pay very close attention to critics. Their ratings tend to boost the sales.
* Game store should be familiar with the top 5 genres. Yet, with regards to the genre, it is less off importance in which region the store is located, since the top 5 genres tend to be the same in na, jp and eu.    
* Given that our dataset is complete, most of the platforms are either "dead" or dying. There seems to be a new trends on the market that are not reflected in our dataset, like "platform free" mobile games (apps) and online games. 
* The avr lifespan of a platform is 7.7 years. This means that buying stocks of a new game of a platform that is 8 years old, might be very risky.  
* All platforms are not born equal. Some do better than others. With that said, the market is constantly changing and moving. Uptrends of yesterdays are downtrends of tommorow. 


<font color='green'><b><u> Reviewer Comment. v. 2  </u></b></font>

<div class="alert alert-success" >
    
**Success:**  The overall conclusion is clear👍

To sum up: an extremely thorough analysis, detailed conclusions and neat style. 

Thank you for good job! 

    
</div>
